In [2]:
import sys
sys.path.append('C:/Users/Kai/Desktop/vis_scBERT/DNABERT_2/')
from get_gene_name_and_seq import *
import pickle

# Get NT sequence from serval patheways of cmiu

In [4]:
raw='''
00190  Oxidative phosphorylation
00680  Methane metabolism
00910  Nitrogen metabolism
00920  Sulfur metabolism
'''
pathway_list = clean_raw_pathway_str(raw, 'cmiu')
cmiu_dic = pathway2NTseq_cmiu(pathway_list)

In [8]:
for k,v in cmiu_dic.items():
    print(k, len(v))

cmiu00190 12
cmiu00680 28
cmiu00910 7
cmiu00920 6


# Build data for "gene sense" task

In [8]:
raw='''
cmiu00010  	Glycolysis / Gluconeogenesis
cmiu00051  	Fructose and mannose metabolism
cmiu00052  	Galactose metabolism
cmiu00500  	Starch and sucrose metabolism
cmiu00520  	Amino sugar and nucleotide sugar metabolism
cmiu00521  	Streptomycin biosynthesis
cmiu01100  	Metabolic pathways
cmiu01110  	Biosynthesis of secondary metabolites
cmiu01120  	Microbial metabolism in diverse environments
cmiu01200  	Carbon metabolism
cmiu01250  	Biosynthesis of nucleotide sugars
'''
pathway_list = clean_raw_pathway_str(raw, 'cmiu')
cmiu_dic = pathway2NTseq_cmiu(pathway_list)

Getting NT sequences: 100%|██████████| 11/11 [03:19<00:00, 18.17s/it]


In [ ]:
cmiu_dic

## TODO: build sentences in neighbors of a gene
- shuffle one single leaf node
- build a larger pseudo dataset

In [4]:
with open("B1H56_02580.pkl", 'rb') as f:
    cmiu_dic = pickle.load(f)

cmiu_dic

{'cmiu00010': {'cmiu:B1H56_02580': 'ATGAAAGGAACGCATATGCTGACTGCAAACGAGTTTATCAGGAAACATAAAATGGAATACACGTCCATTGACGTAAAAAACGAGATGGACACTTACATCGATGAAATGAAAAAGGGATTGTGCGGCGAGAAGAGTTCGCTGCTGATGATCCCTTCCTATATTACGCTGAAAAGCGAAGTGAAGCGGGAAAAACCGGTGATCTGCGTAGACGCGGGCGGGACGAATCTGCGCGTTGCGGTGGCGAAATTTTCCGAGGACGGCTCTTTTTGCACGGAGGAATTCCAGCGGTATTTGATGCCGGGTGTGGAAAAGGAACTTGAGGCCGAAGAATTTTTTGATATTTTGGCGGATTATATCCTGCCTTTTACGAAAATCACGAAAAATATTGTGATTTCATTCGCTTACCGGGCAAAAATACTGCCCGATATCGACTGCGAGATCGTAGAAATCACCAAAGAAGTCAAAGTGAAAAATGCAGGCGGAAAACTGCTGGGCAGAGAGATTTGCGCGGCCTTGGCGAAGAAGGGCGCGGACGGCTGTAATATCGTTGTGGTAAACGACTCCGTGGCAACGGCGCTTGCCGGAAAAGCGGAAAAACTGAACGACGGATACGGCTCCTTTACGGGGACGATACTTGGTACGGGAAGCAACAGCTGCTATATTGAATATATGCGCAACATCGCGAAGCTTGAAGGCGGCGAAGAGGGCCTGATGGTGATTAATACGGAGGCGGGCAGCTATAATAAGGTGCCGCGCAGCGATATCGATATTGCCTATGACGAAAGCACACAAAACCCTGGGATCGGGGTGTTTGAGAAAATGACCAGCGGCGGCTACTTGGGCCCCCTGTGTGATTTTACCCTGCGCACGGCGGCGCGGGAAGGTGTCTTTGAAAAAGGATTTTCCATGGCAGGGACGGTCTCGACCGTCGACGTGAACGCTTTCCTGACGGACGGGTCGGGCG

In [27]:
import requests
from bs4 import BeautifulSoup

def get_tidy_brite(brite_name):
    '''
    *Input*: 
    brite_name: str, e.g. cmiu04131 
    
    *Output*: 
    soup1: str, soup2: dict
    '''
    url = "https://rest.kegg.jp/get/br:" + brite_name
    r = requests.get(url)
    soup1 = BeautifulSoup(r.text, 'html.parser')
    soup1 = soup1.text
    r = requests.get(url+'/json')
    soup2 = r.json()
    return soup1 ,soup2

In [173]:
def find_neighbors(gene_name:str, brite_str:str):
    '''
    **Inputs**:
    - gene_name: str, e.g. B1H56_02580
    - brite_str: str, e.g. brite_str, _ = get_tidy_brite('cmiu00001')
    
    **Outputs**:
    - neighbors: dict, keys are the hierarchies, values are the gene names. 
    hier be like '09100 Metabolism -> 09101 Carbohydrate metabolism -> 00500 Starch and sucrose metabolism [PATH:cmiu00500] -> B1H56_01825 carbohydrate kinase\tK00847 E2.7.1.4; fructokinase [EC:2.7.1.4]'
    '''
    neighbors = {}
    level = ['A','B']
    hier = ''
    names = []
    
    for i in brite_str.split('\n'):
        if i:
            tmp = i[1:].strip().split()
            
            flag = False # flag to indicate whether it is time to update the hier
            if i[0] in ('A', 'B', 'C', 'D', 'E', 'F', 'G'):
                level[1] = i[0]
                flag = True
            if level[1] == 'A':
                hier = i[1:].strip()
            elif flag:
                if level[1]>level[0]:
                    hier += ' -> ' + i[1:].strip()
                    if tmp and tmp[0].startswith(gene_name[:3]):
                        names.append(tmp[0])
                if level[1]==level[0]:
                    # back to the previous level
                    hier = ''.join([i+' -> ' for i in hier.split(' -> ')[:-1]]) + i[1:].strip()
                    if tmp and tmp[0].startswith(gene_name[:3]):
                        names.append(tmp[0])
                if level[1]<level[0]:
                    # back to the previous level twice
                    if gene_name in names:
                        neighbors[hier] = names
                        print(names, hier)
                    hier = ''.join([i+' -> ' for i in hier.split(' -> ')[:-2]]) + i[1:].strip()
                    names = []
            
            # if tmp and tmp[0] == gene_name:
            #     neighbors[hier] = names
                # print(i[1:].strip().split()[0], hier)
                # print(names)
                # print(f'{"-"*10}')
                # print()
            
            level[0] = level[1]
    return neighbors

In [174]:
brite_str, _ = get_tidy_brite('cmiu00001')
a = find_neighbors('B1H56_02580', brite_str)

['B1H56_02580', 'B1H56_05240', 'B1H56_03055', 'B1H56_07930', 'B1H56_02545', 'B1H56_12605', 'B1H56_00960', 'B1H56_11715', 'B1H56_06330', 'B1H56_08575', 'B1H56_05180', 'B1H56_08040', 'B1H56_14070', 'B1H56_00975', 'B1H56_08035', 'B1H56_02480', 'B1H56_05080', 'B1H56_03225', 'B1H56_02845', 'B1H56_07355', 'B1H56_13745', 'B1H56_07350', 'B1H56_00500', 'B1H56_09305', 'B1H56_01335', 'B1H56_01340', 'B1H56_01465', 'B1H56_02180', 'B1H56_02515', 'B1H56_13645', 'B1H56_04770', 'B1H56_09480', 'B1H56_02750', 'B1H56_12425'] 09100 Metabolism -> 09101 Carbohydrate metabolism -> 00010 Glycolysis / Gluconeogenesis [PATH:cmiu00010] -> B1H56_12425 pckA; phosphoenolpyruvate carboxykinase (ATP)	K01610 pckA; phosphoenolpyruvate carboxykinase (ATP) [EC:4.1.1.49]
['B1H56_01825', 'B1H56_13785', 'B1H56_07305', 'B1H56_02710', 'B1H56_07420', 'B1H56_03110', 'B1H56_07460', 'B1H56_08610', 'B1H56_07205', 'B1H56_09465', 'B1H56_11985', 'B1H56_07200', 'B1H56_07415', 'B1H56_03115', 'B1H56_02580', 'B1H56_07930', 'B1H56_02545', 